## Functions for plotting metrics for comparison

> Applies to local experiments not described in the code or documentation.

# Helpers

### Prep

In [ ]:
import sys
import re
import functools
from pathlib import Path
from typing import Optional, Sequence, Union, Any, Callable, Mapping, Tuple

import ast
import numpy as np
import pandas as pd
from tabulate import tabulate
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter
from matplotlib.colors import Normalize
from matplotlib.cm import ScalarMappable
from PIL import Image

DEFAULT_FONTSIZE_LARGE = 18
DEFAULT_FONTSIZE_MEDIUM = 16
DEFAULT_FONTSIZE_SMALL = 13

if 'my_init_done' not in globals():
    my_init_done = True

    try:
        here = Path(__file__).resolve().parent
    except NameError:
        here = Path.cwd()
    sys.path.insert(0, str(here.parent))

def load_styles():
    sns.set_style("ticks", {
        "axes.grid": True,
        "grid.linestyle": "--",
        "grid.alpha": 0.4
    })
    sns.set_context("paper", font_scale=1.0)

    plt.rc('font', family='serif', serif=['Times New Roman', 'Computer Modern'])
    
    plt.rcParams.update({
        "figure.figsize": (8, 4),
        "figure.dpi": 300,
        "savefig.dpi": 300,

        "axes.linewidth": 1.0,
        "axes.labelsize": DEFAULT_FONTSIZE_LARGE,
        "axes.titlepad": 6,

        "xtick.direction": "in",
        "ytick.direction": "in",
        "xtick.major.size": 5,
        "ytick.major.size": 5,
        "xtick.minor.size": 3,
        "ytick.minor.size": 3,
        "xtick.labelsize": DEFAULT_FONTSIZE_MEDIUM,
        "ytick.labelsize": DEFAULT_FONTSIZE_MEDIUM,
        "xtick.bottom": True,
        "xtick.top": False,
        "ytick.left": True,
        "ytick.right": False,

        "lines.linewidth": 1.0,
        "lines.markersize": 6,

        "legend.frameon": False,
        "legend.fontsize": DEFAULT_FONTSIZE_SMALL,
        "legend.loc": "lower right",

        "grid.linewidth": 0.5,
    })
    
load_styles()   # Always load styles for plots

### Tables

In [ ]:
FilterValue = Union[
    Any,
    Sequence[Any],
    Callable[[pd.Series], pd.Series]
]

def _make_mask(df: pd.DataFrame, conds: Mapping[str, FilterValue]) -> pd.Series:
    mask = pd.Series(True, index=df.index)
    for col, cond in conds.items():
        if callable(cond):
            m = cond(df[col])
        elif isinstance(cond, (list, tuple, set, pd.Index)):
            m = df[col].isin(cond)
        else:
            m = df[col] == cond
        mask &= m
    return mask

def filter_df(df: pd.DataFrame, filters: Mapping[str,FilterValue]):
    df2 = df
    masks = [_make_mask(df2, fgroup) for fgroup in filters]
    full_mask = functools.reduce(lambda a,b: a|b, masks)
    df2 = df2.loc[full_mask]
    return df2

def read_csv_numeric_pandas(path: str, sep: str = ';', 
                            list_threshold: float = 0.5, 
                            quotechar: str = '"') -> pd.DataFrame:
    df = pd.read_csv(path, sep=sep, quotechar=quotechar, dtype=str, keep_default_na=False)
    list_re = re.compile(r'^\s*\[.*\]\s*$')
    
    for col in df.columns:
        ser = df[col]
        non_empty = ser[ser.str.strip() != ""]
        if len(non_empty)==0:
            continue
        
        mask = non_empty.str.match(list_re)
        frac = mask.sum() / float(len(non_empty))
        
        if frac >= list_threshold:
            def parse_list_cell(x: str):
                try:
                    lst = ast.literal_eval(x)
                    if not isinstance(lst, (list, tuple)):
                        return x
                    out = []
                    for el in lst:
                        try:
                            out.append(pd.to_numeric(el))
                        except:
                            out.append(el)
                    return out
                except:
                    return x
            df[col] = ser.apply(parse_list_cell)
        else:
            df[col] = pd.to_numeric(ser, errors='ignore')
    return df

def print_pretty_table(df: pd.DataFrame, columns: Optional[Sequence[str]] = None,
                       sort_by: Optional[Union[str, Sequence[str]]] = None,
                       headers: str = "keys", tablefmt: str = "psql",
                       showindex: bool = False) -> None:
    df2 = df

    if columns is not None:
        df2 = df2.loc[:, columns]
        
    if sort_by is not None:
        df2 = df2.sort_values(by=sort_by, ascending=True)
    
    s = tabulate(df2, headers=headers, tablefmt=tablefmt, showindex=showindex,
                 floatfmt=".4g")
    print(s)

### Line plots (x2)

In [ ]:
NumberFormatter = Callable[[float], str]

def plot_2lines(df: pd.DataFrame, x: str, y1: str, y2: str, *,
                y1_label: Optional[str] = None, y2_label: Optional[str] = None,
                xlabel: Optional[str] = None, ylabel: Optional[str] = None,
                y_multiplier: float = 1.0, y_suffix: str = "",
                ylim: Optional[Tuple[float, float]] = None,
                legend_loc: str = "best", fmt: Optional[NumberFormatter] = None,
                annotate: bool = False, line_palette: Union[str, Sequence] = "tab10",
                markers: Optional[Sequence[str]] = None, save_path: Optional[str] = None,
                show: bool = True) -> Tuple[plt.Figure, plt.Axes]:
    df2 = df[[x, y1, y2]].dropna().sort_values(by=x)
    xs = df2[x].values
    y1s = df2[y1].values
    y2s = df2[y2].values

    def _default_fmt(v: float) -> str:
        s = f"{v:.2f}".replace(".", ",")
        return f"{s}{y_suffix}"
    fmt = fmt or _default_fmt
    def _wrapped_fmt(v: float) -> str:
        return fmt(v * y_multiplier)

    if isinstance(line_palette, str):
        colors = sns.color_palette(line_palette, 2)
    else:
        colors = list(line_palette)
    default_markers = ["o", "s"]
    if markers is None:
        markers = default_markers
    elif len(markers) < 2:
        raise ValueError("You need at least 2 markers")

    fig, ax = plt.subplots()
    ax.plot(xs, y1s, color=colors[0], marker=markers[0], linewidth=2, markersize=4, label=y1_label or y1)
    ax.plot(xs, y2s, color=colors[1], marker=markers[1], linewidth=2, markersize=4, label=y2_label or y2)

    if annotate:
        for xi, yi in zip(xs, y1s):
            ax.annotate(_wrapped_fmt(yi), xy=(xi, yi), xytext=(0,5),
                        textcoords="offset points", ha="center", va="bottom")
        for xi, yi in zip(xs, y2s):
            ax.annotate(_wrapped_fmt(yi), xy=(xi, yi), xytext=(0,5),
                        textcoords="offset points", ha="center", va="bottom")

    if xlabel: ax.set_xlabel(xlabel)
    if ylabel: ax.set_ylabel(ylabel)
    
    xt = sorted(df2[x].unique())
    if len(xt) > 20:
        start, end = xt[0], xt[-1]
        step = 10
        ticks = [start]
        first = ((start // step) + 1) * step
        ticks += list(range(first, end + 1, step))
        ax.set_xticks(ticks)
    else:
        ax.set_xticks(xt)

    if ylim:
        ax.set_ylim(*ylim)
    else:
        y0, y1_ = min(y1s.min(), y2s.min()), max(y1s.max(), y2s.max())
        pad = (y1_ - y0) * 0.1 if y1_ != y0 else 1
        ax.set_ylim(y0 - pad, y1_ + pad)

    ax.yaxis.set_major_formatter(FuncFormatter(lambda v, pos: _wrapped_fmt(v)))
    ax.legend(frameon=False, loc=legend_loc)

    sns.despine(top=True, right=True)
    fig.tight_layout()
    if save_path:
        fig.savefig(save_path)
    if show:
        plt.show()
    return fig, ax

### Line plots (x2xN)

In [ ]:
NumberFormatter = Callable[[float], str]

def plot_2Nlines(df: pd.DataFrame, x: str, y1: str, y2: str, param_col: str, param_val, *,
                 quantiles: Tuple[float, float] = (0, 1), y1_label: Optional[str] = None,
                 y2_label: Optional[str] = None, xlabel: Optional[str] = None, ylabel: Optional[str] = None,
                 y_multiplier: float = 1.0, y_suffix: str = "", ylim: Optional[Tuple[float, float]] = None,
                 legend_loc: str = "best", fmt: Optional[NumberFormatter] = None, annotate: bool = False,
                 line_palette: Union[str, Sequence] = "tab10", markers: Optional[Sequence[str]] = None,
                 save_path: Optional[str] = None, show: bool = True) -> Tuple[plt.Figure, plt.Axes]:
    runs_x = []
    runs_y1 = []
    runs_y2 = []
    main_x = main_y1 = main_y2 = None

    for _, row in df.iterrows():
        try:
            xv = ast.literal_eval(row[x]) if isinstance(row[x], str) else row[x]
            y1v = ast.literal_eval(row[y1]) if isinstance(row[y1], str) else row[y1]
            y2v = ast.literal_eval(row[y2]) if isinstance(row[y2], str) else row[y2]
        except Exception as e:
            raise ValueError(f"Failed to parse row {_}: {e}")

        xv = np.array(xv, dtype=float)
        y1v = np.array(y1v, dtype=float)
        y2v = np.array(y2v, dtype=float)

        if row[param_col] == param_val:
            main_x, main_y1, main_y2 = xv, y1v, y2v

        runs_x.append(xv)
        runs_y1.append(y1v)
        runs_y2.append(y2v)

    if main_x is None:
        raise ValueError(f"Row not found with {param_col} == {param_val}")

    for arr in runs_x:
        if arr.shape != main_x.shape or not np.allclose(arr, main_x):
            raise ValueError("All series must have the same x-points and their order.")

    Y1 = np.stack(runs_y1, axis=0)
    Y2 = np.stack(runs_y2, axis=0)

    if isinstance(line_palette, str):
        colors = sns.color_palette(line_palette, 2)
    else:
        colors = list(line_palette)

    default_markers = ["o", "s"]
    if markers is None:
        markers = default_markers
    elif len(markers) < 2:
        raise ValueError("You need at least 2 markers")

    def _default_fmt(v: float) -> str:
        s = f"{v:.2f}".replace(".", ",")
        return f"{s}{y_suffix}"
    fmt = fmt or _default_fmt
    def _wrapped_fmt(v: float) -> str:
        return fmt(v * y_multiplier)

    fig, ax = plt.subplots()

    q_low, q_high = quantiles

    lower1 = np.quantile(Y1, q_low, axis=0)
    upper1 = np.quantile(Y1, q_high, axis=0)
    ax.fill_between(main_x, lower1, upper1, color=colors[0], alpha=0.2, linewidth=0)

    lower2 = np.quantile(Y2, q_low, axis=0)
    upper2 = np.quantile(Y2, q_high, axis=0)
    ax.fill_between(main_x, lower2, upper2, color=colors[1], alpha=0.2, linewidth=0)

    ax.plot(main_x, main_y1,
            color=colors[0], marker=markers[0],
            linewidth=2, markersize=5,
            label=y1_label or y1)
    ax.plot(main_x, main_y2,
            color=colors[1], marker=markers[1],
            linewidth=2, markersize=5,
            label=y2_label or y2)

    if annotate:
        for xi, yi in zip(main_x, main_y1):
            ax.annotate(_wrapped_fmt(yi), xy=(xi, yi),
                        xytext=(0, 5), textcoords="offset points",
                        ha="center", va="bottom")
        for xi, yi in zip(main_x, main_y2):
            ax.annotate(_wrapped_fmt(yi), xy=(xi, yi),
                        xytext=(0, 5), textcoords="offset points",
                        ha="center", va="bottom")

    if xlabel:
        ax.set_xlabel(xlabel)
    if ylabel:
        ax.set_ylabel(ylabel)

    unique_x = main_x
    if len(unique_x) > 20:
        start, end = unique_x[0], unique_x[-1]
        step = 10
        ticks = [start]
        first = ((start // step) + 1) * step
        ticks += list(range(int(first), int(end) + 1, step))
        ax.set_xticks(ticks)
    else:
        ax.set_xticks(unique_x)

    if ylim:
        ax.set_ylim(*ylim)
    else:
        ymin = min(lower1.min(), lower2.min())
        ymax = max(upper1.max(), upper2.max())
        pad = (ymax - ymin) * 0.1 if ymax != ymin else 1
        ax.set_ylim(ymin - pad, ymax + pad)

    ax.yaxis.set_major_formatter(FuncFormatter(lambda v, pos: _wrapped_fmt(v)))
    ax.legend(frameon=False, loc=legend_loc)
    sns.despine(top=True, right=True)
    fig.tight_layout()

    if save_path:
        fig.savefig(save_path)
    if show:
        plt.show()

    return fig, ax

### Bar Plots (x1)

In [ ]:
NumberFormatter = Callable[[float], str]

def plot_bar(df: pd.DataFrame, x: str, y: str, *,
             kind: str = "bar", hue: Optional[str] = None,                       
             palette: Union[str, Sequence] = "tab10", cmap: str = "viridis",                        
             xlabel: Optional[str] = None, ylabel: Optional[str] = None,
             hue_label: Optional[str] = None, y_multiplier: float = 1.0,
             y_suffix: str = "", ylim: Optional[Tuple[float, float]] = None,
             pallim: Optional[Tuple[float, float]] = None,
             fmt: Optional[NumberFormatter] = None, annotate: bool = True,
             width: float = 0.8, save_path: Optional[str] = None,
             show: bool = True) -> Tuple[plt.Figure, plt.Axes]:
    cols = [x, y] + ([hue] if hue is not None else [])
    df2 = df[cols].dropna().copy()
    df2 = df2.sort_values(by=x)
    xs = df2[x].values
    ys = df2[y].values

    def _default_fmt(v: float) -> str:
        s = f"{v:.2f}".replace(".", ",")
        return f"{s}{y_suffix}"
    fmt = fmt or _default_fmt
    def _wrapped_fmt(raw_v: float) -> str:
        return fmt(raw_v * y_multiplier)

    bar_colors = None
    color_lut = None

    if hue is not None and hue in df2.columns:
        vals = df2[hue]
        if pd.api.types.is_numeric_dtype(vals):
            print(f"[{vals.min()}, {vals.max()}]")
            if pallim is None:
                pallim = [vals.min(), vals.max()]
            norm = Normalize(vmin=pallim[0], vmax=pallim[1])
            cmap_obj = plt.get_cmap(cmap)
            bar_colors = cmap_obj(norm(vals.values))
        else:
            cats = vals.unique().tolist()
            pal = sns.color_palette(palette, len(cats))
            color_lut = dict(zip(cats, pal))
            bar_colors = vals.map(color_lut).tolist()
    else:
        bar_colors = sns.color_palette(palette, len(xs))

    legend_title = hue_label or hue
    fig, ax = plt.subplots()

    if kind == "bar":
        bars = ax.bar(
            xs, ys,
            color=bar_colors,
            edgecolor="black",
            width=width,
            zorder=1
        )

        for sp in ax.spines.values():
            sp.set_zorder(2)
            sp.set_clip_on(False)

        xmin, xmax = xs.min(), xs.max()
        pad = width * 1.0
        ax.set_xlim(xmin - pad, xmax + pad)

        if color_lut is not None:
            import matplotlib.patches as mpatches
            handles = [
                mpatches.Patch(color=color_lut[c], label=str(c))
                for c in color_lut
            ]
            ax.legend(handles=handles,
                      title=legend_title, 
                      loc="best")

        if pd.api.types.is_numeric_dtype(df2[hue]) if hue is not None else False:
            sm = ScalarMappable(norm=norm, cmap=cmap_obj)
            sm.set_array([])
            cbar = fig.colorbar(sm, ax=ax,
                                fraction=0.046, pad=0.04)
            cbar.set_label(legend_title) 
            cbar.ax.tick_params(labelsize=DEFAULT_FONTSIZE_SMALL)
    else:
        line_color = bar_colors[0] if isinstance(bar_colors, (list, tuple)) else bar_colors
        ax.plot(xs, ys, color=line_color, marker="o", linewidth=2)
        ax.margins(x=0.05)

    if annotate and kind == "bar":
        for bar in bars:
            h = bar.get_height()
            ax.annotate(
                _wrapped_fmt(h),
                xy=(bar.get_x() + bar.get_width() / 2, h),
                xytext=(0, 5),
                textcoords="offset points",
                ha="center", va="bottom",
                fontsize=DEFAULT_FONTSIZE_SMALL
            )
    elif annotate:
        for xi, yi in zip(xs, ys):
            ax.annotate(
                _wrapped_fmt(yi),
                xy=(xi, yi),
                xytext=(0, 5),
                textcoords="offset points",
                ha="center", va="bottom",
                fontsize=DEFAULT_FONTSIZE_SMALL
            )

    if xlabel:
        ax.set_xlabel(xlabel, labelpad=10)
    if ylabel:
        ax.set_ylabel(ylabel, labelpad=10)

    ax.set_xticks(xs)
    if ylim is not None:
        ax.set_ylim(*ylim)
    else:
        ymin0, ymax0 = ys.min(), ys.max()
        ypad = (ymax0 - ymin0) * 0.1 if ymax0 > ymin0 else 1
        ax.set_ylim(ymin0 - ypad, ymax0 + ypad)

    ax.yaxis.set_major_formatter(FuncFormatter(lambda val, pos: _wrapped_fmt(val)))

    sns.despine(top=True, right=True)
    fig.tight_layout()

    if save_path:
        fig.savefig(save_path)
    if show:
        plt.show()

### Line Plots (x3)

In [ ]:
NumberFormatter = Callable[[float], str]

def plot_3lines_hue(df: pd.DataFrame, x: str, y: str, z: str, hue: str, *,
                    z_label: Optional[str] = None, hue_label: Optional[str] = None,           
                    xlabel: Optional[str] = None, ylabel: Optional[str] = None,
                    y_multiplier: float = 1.0, y_suffix: str = "",
                    ylim: Optional[Tuple[float, float]] = None,
                    pallim: Optional[Tuple[float, float]] = None,
                    fmt: Optional[NumberFormatter] = None,
                    annotate: bool = False, palette: Union[str, Sequence] = "tab10",     
                    cmap: str = "viridis", markers: Optional[Sequence[str]] = None,
                    line_palette: Union[str, Sequence] = "tab10",
                    save_path: Optional[str] = None,
                    show: bool = True) -> Tuple[plt.Figure, plt.Axes]:
    if z not in df.columns or hue not in df.columns:
        raise KeyError(f"Missing column z= '{z} 'or hue=' {hue}'")
    df2 = df[[x, y, z, hue]].dropna().sort_values(by=x).copy()

    def _default_fmt(v: float) -> str:
        s = f"{v:.2f}".replace(".", ",")
        return f"{s}{y_suffix}"
    fmt = fmt or _default_fmt
    def _wrapped_fmt(raw_v: float) -> str:
        return fmt(raw_v * y_multiplier)

    is_num = pd.api.types.is_numeric_dtype(df2[hue])
    if is_num:
        print(f"[{df2[hue].min()}, {df2[hue].max()}]")
        if pallim is None:
            pallim = [df2[hue].min(), df2[hue].max()]
        norm     = Normalize(vmin=pallim[0], vmax=pallim[1])
        cmap_obj = plt.get_cmap(cmap)
    else:
        cats = df2[hue].unique().tolist()
        pal  = sns.color_palette(palette, len(cats))
        lut  = dict(zip(cats, pal))

    groups = df2[z].unique().tolist()
    ng     = len(groups)

    default_markers = ["o","s","^","D","v","<",">","p","X","*"]
    if markers is None:
        markers = (default_markers * ((ng//len(default_markers))+1))[:ng]
    elif len(markers) < ng:
        raise ValueError(f"You need at least {ng} markers")

    line_colors = sns.color_palette(line_palette, ng)

    fig, ax = plt.subplots()
    for i, grp in enumerate(groups):
        sub = df2[df2[z] == grp]
        xs, ys = sub[x].values, sub[y].values

        ax.plot(xs, ys,
                color=line_colors[i],
                linewidth=1.8,
                zorder=1)

        if is_num:
            ax.scatter(xs, ys,
                       c=sub[hue].values,
                       cmap=cmap, norm=norm,
                       marker=markers[i],
                       edgecolor='black',
                       s=50,
                       zorder=2)
        else:
            ax.scatter(xs, ys,
                       color=[lut[val] for val in sub[hue]],
                       marker=markers[i],
                       edgecolor='black',
                       s=50,
                       zorder=2)

        if annotate:
            for xi, yi in zip(xs, ys):
                ax.annotate(_wrapped_fmt(yi),
                            xy=(xi, yi),
                            xytext=(0, 5),
                            textcoords="offset points",
                            ha="center", va="bottom",
                            fontsize=DEFAULT_FONTSIZE_SMALL)

    if xlabel: ax.set_xlabel(xlabel, labelpad=8)
    if ylabel: ax.set_ylabel(ylabel, labelpad=8)

    xt = sorted(df2[x].unique())
    ax.set_xticks(xt)

    if ylim:
        ax.set_ylim(*ylim)
    else:
        y0, y1 = df2[y].min(), df2[y].max()
        pad = (y1 - y0)*0.1 if y1!=y0 else 1
        ax.set_ylim(y0-pad, y1+pad)

    ax.yaxis.set_major_formatter(FuncFormatter(lambda v,pos: _wrapped_fmt(v)))

    ztitle = z_label or z
    for i, grp in enumerate(groups):
        ax.plot([], [],
                color=line_colors[i],
                marker=markers[i],
                linestyle='-',
                linewidth=1.8,
                markersize=6,
                label=str(grp).upper())
    leg = ax.legend(title=ztitle, frameon=False, loc='lower right')
    title = leg.get_title()
    title.set_fontsize(DEFAULT_FONTSIZE_SMALL)

    if is_num:
        sm = ScalarMappable(norm=norm, cmap=cmap_obj)
        sm.set_array([])
        cbar = fig.colorbar(sm, ax=ax, fraction=0.046, pad=0.04)
        cbar.set_label(hue_label or hue)

    sns.despine(top=True, right=True)
    fig.tight_layout()
    if save_path:
        fig.savefig(save_path)
    if show:
        plt.show()

### Confusion Matrix

In [ ]:
def plot_confusion_matrix(df: pd.DataFrame, actual: str, predicted: str, *,
                          label_map: Optional[pd.DataFrame] = None,
                          labels: Optional[Sequence[Union[int, str]]] = None,
                          normalize: Optional[str] = None,
                          cmap: str = "Blues",
                          annot: bool = True,
                          fmt: str = "",
                          cbar: bool = True,
                          cbar_label: Optional[str] = None,
                          xlabel: Optional[str] = None,
                          ylabel: Optional[str] = None,
                          pallim: Optional[Tuple[float, float]] = None,
                          figsize: Tuple[int, int] = (8, 6),
                          count_col: str = "count",
                          save_path: Optional[str] = None,
                          show: bool = True) -> Tuple[plt.Figure, plt.Axes]:
    df2 = df.copy()
    
    if label_map is not None:
        lm = label_map.set_index('label_id')['label']
        df2[actual] = df2[actual].map(lm)
        df2[predicted] = df2[predicted].map(lm)
        
    if labels is None:
        a = sorted(df2[actual].dropna().unique().tolist())
        p = sorted(df2[predicted].dropna().unique().tolist())
        labels = sorted(set(a) | set(p))
        
    if count_col in df2.columns:
        cm = df2.pivot_table(index=actual, columns=predicted, values=count_col,
                             aggfunc='sum', fill_value=0)
    else:
        cm = pd.crosstab(df2[actual], df2[predicted], dropna=False)    
    cm = cm.reindex(index=labels, columns=labels, fill_value=0)
    
    mat = cm.values.astype(int)
    if normalize in ("true", "pred", "all"):
        if normalize == "true":
            mat = mat / mat.sum(axis=1, keepdims=True)
        elif normalize == "pred":
            mat = mat / mat.sum(axis=0, keepdims=True)
        else:
            mat = mat / mat.sum()
            
    fig, ax = plt.subplots(figsize=figsize)
    
    if pallim is not None:
        vmin, vmax = pallim
    elif normalize is not None:
        vmin, vmax = 0.0, 1.0
    else:
        vmin, vmax = None, None
        
    sns.heatmap(mat, ax=ax, cmap=cmap, annot=annot, fmt=fmt, cbar=cbar,
                cbar_kws={'label': cbar_label} if cbar_label else {},
                xticklabels=labels, yticklabels=labels, vmin=vmin, vmax=vmax)
    
    if cbar and cbar_label:
        cb = ax.collections[0].colorbar
        cb.set_label(cbar_label)
        
    ax.set_xlabel(xlabel or predicted)
    ax.set_ylabel(ylabel or actual)

    fig.tight_layout()
    if save_path:
        fig.savefig(save_path)
    if show:
        plt.show()
        
    return fig, ax

### Examples

In [ ]:
def plot_six_images_from_paths(image_paths: Sequence[Sequence[str]],
                               img_labels: Sequence[Sequence[str]],
                               col_labels: Sequence[str],
                               figsize: Tuple[float, float] = (4, 6),
                               dpi: int = 400, cmap: str = 'gray',
                               target_size: Optional[Tuple[int, int]] = (28, 28),
                               interpolation: str = 'nearest',
                               save_path: Optional[str] = None,
                               show: bool = True) -> Tuple[plt.Figure, np.ndarray]:
    fig = plt.figure(figsize=figsize, dpi=dpi)
    gs = fig.add_gridspec(
        3, 2,
        left=0.08, right=0.98,
        top=0.92, bottom=0.05,
        hspace=0.25, wspace=0.15
    )

    axes = np.empty((3, 2), dtype=object)
    for i in range(3):
        for j in range(2):
            axes[i, j] = fig.add_subplot(gs[i, j])
            
    for col in (0, 1):
        x = 0.08 + (0.98 - 0.08) * (col * 0.5 + 0.25)
        fig.text(x, 0.96, col_labels[col], ha='center', va='bottom', fontsize=12)
        
    for col in (0, 1):
        for row in range(3):
            path = image_paths[col][row]
            pil = Image.open(path)
            if target_size is not None:
                resample = Image.NEAREST if interpolation == 'nearest' else Image.BILINEAR
                pil = pil.resize(target_size, resample=resample)
            img = np.array(pil)
            ax = axes[row, col]
            if img.ndim == 2:
                ax.imshow(img, cmap=cmap)
            else:
                ax.imshow(img)
            ax.set_title(img_labels[col][row], pad=5, fontsize=9)
            ax.axis('off')
            
    if save_path:
        fig.savefig(save_path, dpi=dpi, bbox_inches='tight', pad_inches=0)
        
    if show:
        plt.show()
        
    return fig, axes